# A. Process data and fill in missing values

In [8]:
import pandas as pd
import numpy as np
data = pd.read_csv("adj_close_hist.csv")
print("Total rows: {0}".format(len(data)))
print(list(data))

Total rows: 1793
['date', 'A', 'AMZN', 'CAT', 'GDP', 'MSFT', 'NFLX', 'TSLA', 'XOM']


In [9]:
data.fillna(0)
filled_data = data.fillna(method='pad')

In [10]:
arr_data = np.array(filled_data)
arr_data

array([['2011-01-03', 28.572239933249, 184.22, ..., 25.487142857142853,
        26.62, 60.415733100366],
       ['2011-01-04', 28.306166065676, 185.01, ..., 25.91, 26.67,
        60.699375039805005],
       ['2011-01-05', 28.244764403928, 187.42, ..., 25.67571428571428,
        26.83, 60.537293931554004],
       ...,
       ['2018-02-14', 69.7, 1451.05, ..., 266.0, 322.31, 76.46],
       ['2018-02-15', 72.02, 1461.76, ..., 280.27, 334.065, 76.21],
       ['2018-02-16', 71.97, 1448.69, ..., 278.52, 335.49, 76.54]],
      dtype=object)

# B. Implement trading strategies

In [11]:
#calculating SMA
def SMA(x,n,col):
    avgs = []
    for i in range(len(x)-n+1):
            avgs.append(sum(x[i:i+n,col])/n)
    return avgs

#print(len(SMA(arr_data,5,1)))

#initial position: long
#getting position cash flows
def position_cf(x,n,N,col):
    position = []
    SMA_n = SMA(x,n,col)[N-n:]
    SMA_N = SMA(x,N,col)

    #comparing shor-term and long-term SMA
    for i in range(len(x)-N+1):
        if SMA_n[i] > SMA_N[i]:
            position.append(-x[i+N-1,col])
        if SMA_n[i] < SMA_N[i]:
            position.append(x[i+N-1,col])
        elif SMA_n[i] == SMA_N[i]:
            position.append(position[i-1])
    return position

In [12]:
#getting transaction cash flows
def transaction_cf(x,n,N,col):
    tran=[]
    p = position_cf(arr_data,n,N,col)
    tran.append(p[0])
    #Buy and hold until SMA_n < SMA_N and then sell it  
    for j in range(0,len(p)):
        if ((p[j]*p[j-1])<0) & (p[j-1]<0):
            tran.append(p[j])
        if ((p[j]*p[j-1])<0) & (p[j-1]>0):
            tran.append(p[j])
    return tran

# C. Comapre performance statistics for the trading strategies

# a. Average position turn-over per annum

In [6]:
import datetime as dt
from datetime import timedelta

def avg_turnover_per_ann(x,n,N,col):
    date = []
    nyears = 0
    tran_num = 0
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))
        
    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
    tran_num = len(transaction_cf(x,n,N,col))/2
    return tran_num/nyears

In [7]:
avg_turnover_per_ann(arr_data,1,5,1)

33.46885813150353

# b. Average position holding period

In [13]:
def avg_hp(x,n,N,col):
    trandate_index=[]
    p = position_cf(x,n,N,col)
    trandate_index=[]
    trandate=[]
    hp_sum=0
    trandate_index.append(N-1)
    date = []
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))
        
    #Buy and hold until SMA_n < SMA_N and then sell it  
    for j in range(0,len(p)):
        if ((p[j]*p[j-1])<0) & (p[j-1]<0):
            trandate_index.append(j+N-1)
        if ((p[j]*p[j-1])<0) & (p[j-1]>0):
            trandate_index.append(j+N-1)
            
    for i in range(0,len(trandate_index)-1,2):
        hp_sum += (date[trandate_index[i+1]]-date[trandate_index[i]]).days        

    return hp_sum/(len(transaction_cf(x,n,N,col))/2)

avg_hp(arr_data,1,5,1)

6.20125786163522

# c. Average annualized return

In [22]:
def avg_ann_ret(x,n,N,col,principal):
    shares = 0
    profit = 0
    rets = []
    total_ret = 1
    date = []
    nyears = 0
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))

    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
       
    a = transaction_cf(x,n,N,col)
    for i in range(0,len(a)-1,2):
        shares = principal/(abs(a[i]))
        profit = (a[i+1] + a[i])*shares
        rets.append(profit/principal)
        principal += profit

    for i in range(len(rets)):
        total_ret *= (rets[i]+1)
    
    return total_ret**(1/nyears)-1

avg_ann_ret(arr_data,1,5,1,10000)

0.02038624040820869

# d. Average annualized return volatility

In [15]:
def avg_ann_ret_vol(x,n,N,col,principal):
    shares = 0
    profit = 0
    rets = []
    a = transaction_cf(x,n,N,col)
    date = []
    nyears = 0
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))

    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
    
    for i in range(0,len(a)-1,2):
        shares = principal/(abs(a[i]))
        profit = (a[i+1] + a[i])*shares
        rets.append(profit/principal)
        principal += profit
        
    return np.var(rets)/nyears

avg_ann_ret_vol(arr_data,1,5,1,10000)

0.00012569067029363396

# Summary: Outputs for all stocks

In [28]:
for i in range(1,arr_data.shape[1]):
    print('====================',list(data)[i],'====================')
    print('Short-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,1,5,i))
    print('Medium-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,15,50,i))
    print('Long-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,50,200,i),'\n')
    
    print('Short-term average position holding period:',avg_hp(arr_data,1,5,i))
    print('Medium-term average position holding period:',avg_hp(arr_data,15,50,i))
    print('Long-term average position holding period:',avg_hp(arr_data,50,200,i),'\n')
    
    print('Short-term average annualized return:',avg_ann_ret(arr_data,1,5,i,10000))
    print('Medium-term average annualized return:',avg_ann_ret(arr_data,15,50,i,10000))
    print('Long-term average annualized return:',avg_ann_ret(arr_data,50,200,i,10000),'\n')
    
    print('Short-term average annualized return volatility:',avg_ann_ret_vol(arr_data,1,5,i,10000))
    print('Medium-term average annualized return volatility:',avg_ann_ret_vol(arr_data,15,50,i,10000))
    print('Long-term average annualized return volatility:',avg_ann_ret_vol(arr_data,50,200,i,10000),'\n')

==================== A ====================
Short-term average position turn-over per annum: 33.46885813150353
Medium-term average position turn-over per annum: 2.8767781622466346
Long-term average position turn-over per annum: 0.9121491733952743 

Short-term average position holding period: 6.20125786163522
Medium-term average position holding period: 75.3170731707317
Long-term average position holding period: 151.23076923076923 

Short-term average annualized return: 0.02038624040820869
Medium-term average annualized return: 0.08213912760527031
Long-term average annualized return: 0.13488182442902796 

Short-term average annualized return volatility: 0.00012569067029363396
Medium-term average annualized return volatility: 0.0016807310036209768
Long-term average annualized return volatility: 0.08238262637738744 

==================== AMZN ====================
Short-term average position turn-over per annum: 32.90753556326028
Medium-term average position turn-over per annum: 2.59611687